# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tasiilaq,65.6145,-37.6368,-0.91,97,100,9.39,GL,1670106070
1,1,punta arenas,-53.1500,-70.9167,10.06,46,20,11.83,CL,1670106484
2,2,quelimane,-17.8786,36.8883,25.97,73,20,2.57,MZ,1670106078
3,3,ushuaia,-54.8000,-68.3000,6.81,65,75,8.75,AR,1670105897
4,4,villa union,23.2000,-106.2333,28.98,45,0,2.57,MX,1670106586


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    color = 'City',
    size = 'Humidity',
    scale = 0.75,
    frame_width = 700,
    frame_height = 500,
    alpha = .5
)
# Display the map plot_1
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
preferred_city = city_data_df.loc[(city_data_df['Max Temp'] >= 20) & (city_data_df['Max Temp'] <= 30) & (city_data_df['Humidity'] > 30) & (city_data_df['Humidity'] < 60) & (city_data_df['Cloudiness'] < 50)]
# Drop any rows with null values
clean_preferred_city = preferred_city.dropna(how='any')
# Display sample data
clean_preferred_city.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,villa union,23.2000,-106.2333,28.98,45,0,2.57,MX,1670106586
9,9,tarata,-17.6167,-66.0167,20.68,48,44,4.30,BO,1670106586
45,45,bandar-e lengeh,26.5579,54.8807,22.95,53,0,0.96,IR,1670106592
102,102,mildura,-34.2000,142.1500,27.26,48,0,5.36,AU,1670106122
103,103,navrongo,10.8956,-1.0921,28.07,40,31,1.40,GH,1670106602


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_preferred_city[['City', 'Country', 'Lat', 'Lng', 'Humidity']]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']= ""
# Display sample data
hotel_df

/var/folders/r1/w1303vn5085bcrsmp_83wz2r0000gn/T/ipykernel_50040/152995489.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name']= ""


,City,Country,Lat,Lng,Humidity,Hotel Name
4,villa union,MX,23.2000,-106.2333,45,
9,tarata,BO,-17.6167,-66.0167,48,
45,bandar-e lengeh,IR,26.5579,54.8807,53,
102,mildura,AU,-34.2000,142.1500,48,
103,navrongo,GH,10.8956,-1.0921,40,
128,kahului,US,20.8947,-156.4700,59,
132,sesheke,ZM,-17.4759,24.2968,31,
158,tecoanapa,MX,16.5167,-98.7500,57,
168,graciano sanchez,MX,23.9167,-99.2833,58,
170,tocopilla,CL,-22.0920,-70.1979,58,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
lon = 0
lat = 0
limit = 1

filters = f'circle:{lon},{lat},{radius}'
bias = f"proximity:{lon},{lat}"
api_params = {
    "categories":"accommodation.hotel",
    "filter": filters,
    "bias": bias,
    "limit": limit,
    "apiKey": geoapify_key  
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = hotel_df.loc[index,'Lng']
    lat = hotel_df.loc[index,'Lat']

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    api_params["filter"] = f'circle:{lon},{lat},{radius}'
    api_params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = api_params).json()
    
#     # Convert the API response to JSON format
#     name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
villa union - nearest hotel: No hotel found
tarata - nearest hotel: Centro Alternativo Tara Tara
bandar-e lengeh - nearest hotel: Old owner do not like foreigners hotel
mildura - nearest hotel: Chaffey Motor Inn
navrongo - nearest hotel: Mayaga
kahului - nearest hotel: Maui Seaside Hotel
sesheke - nearest hotel: Sisheke Lodge
tecoanapa - nearest hotel: No hotel found
graciano sanchez - nearest hotel: No hotel found
tocopilla - nearest hotel: hotel Isidora
monte patria - nearest hotel: Hotel Monte Cordillera
la palma - nearest hotel: La Quinta Inn & Suites by Wyndham Buena Park
pasni - nearest hotel: No hotel found
dwarka - nearest hotel: The Dwarika Hotel
hwange - nearest hotel: No hotel found
kapiri mposhi - nearest hotel: Chanika Lodge
stawell - nearest hotel: Coorrabin Motor Inn
thabazimbi - nearest hotel: Maroela Guest Lodge
mount gambier - nearest hotel: The Old Mount Gambier Gaol
srivardhan - nearest hotel: No hotel found
charters towers - nearest hotel: No 

,City,Country,Lat,Lng,Humidity,Hotel Name
4,villa union,MX,23.2000,-106.2333,45,No hotel found
9,tarata,BO,-17.6167,-66.0167,48,Centro Alternativo Tara Tara
45,bandar-e lengeh,IR,26.5579,54.8807,53,Old owner do not like foreigners hotel
102,mildura,AU,-34.2000,142.1500,48,Chaffey Motor Inn
103,navrongo,GH,10.8956,-1.0921,40,Mayaga
128,kahului,US,20.8947,-156.4700,59,Maui Seaside Hotel
132,sesheke,ZM,-17.4759,24.2968,31,Sisheke Lodge
158,tecoanapa,MX,16.5167,-98.7500,57,No hotel found
168,graciano sanchez,MX,23.9167,-99.2833,58,No hotel found
170,tocopilla,CL,-22.0920,-70.1979,58,hotel Isidora


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:

hotel_df_plot = hotel_df.hvplot.points(
    x = "Lng",
    y = "Lat",
    geo = True,
    tiles = "OSM",
    color = 'City',
    scale = 0.75,
    size = "Humidity",
    frame_width = 700,
    frame_height = 500,
    alpha = .5,
    hover_cols = ['Hotel Name', 'City', 'Country'])


# Display the map plot_1
hotel_df_plot


/opt/anaconda3/lib/python3.9/site-packages/cartopy/crs.py:245: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(multi_line_string) > 1:
/opt/anaconda3/lib/python3.9/site-packages/cartopy/crs.py:297: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for line in multi_line_string:
/opt/anaconda3/lib/python3.9/site-packages/cartopy/crs.py:364: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(p_mline) > 0:
/opt/anaconda3/lib/python3.9/site-packages/cartopy/crs.py:245: ShapelyDeprecationWarning: __len__

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)